In [1]:
import numpy as np
import Utilityfunctions as utils
import jax.numpy as jnp
import ssr_kronvec_jax as ssr
import ssr_likelihood_jax as jax_lik
import ssr_likelihood as pyt

In [2]:
n = 3
theta = utils.random_theta(n, 0.0)
n_dat = 100
dat = np.random.binomial(n=1, p=0.6, size=n_dat*(2*n+1))
dat = jnp.array(dat).reshape((n_dat, -1))
dat = dat.at[:,-1].set(1)
dat = jnp.column_stack((dat, jnp.ones(n_dat, dtype=int)))
lam1 = 0.5
lam2 = 1.5

In [ ]:
#dat_point = dat.at[0,0:2*n+1].get()
#m = dat_point.sum()
#p0 = np.zeros(2**m)
#p0[0] = 1
#pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)

In [ ]:
#dat_point = jnp.array([1,1,0,0,1,1,1], dtype = int)
#m = dat_point.sum()
#p0 = np.zeros(2**m)
#p0[0] = 1
#pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)
#count = dat_point[1:2*n:2].sum()
#latent_dist = jnp.zeros(2**(count))
#print(utils.ssr_obs_dist(pTh1, dat_point, n, obs_prim=True))
#inds = ssr.obs_inds(pTh1, dat_point, latent_dist, obs_prim=True)
#utils.trunk_states(dat_point)[2**(m-1)+inds[0].at[0].get()]

In [ ]:
#jax_lik.log_prob_coupled(dat, theta, lam1, lam2)

In [ ]:
#jax_lik.log_prob_single(dat, jnp.array(theta), lam1)

In [ ]:
#jax_lik.grad_single(dat, jnp.array(theta), lam1)

In [11]:
state = jnp.array([0,1, 1,1, 0,1, 1]) #dat.at[2, 0:2*n+1].get()
m = int(state.sum())
latent_state = np.append(state.at[1:2*n:2].get(), state.at[-1].get())
count = latent_state.sum()
print(state)
print(latent_state)
latent_dist = jnp.zeros(2**count)
p0 = jnp.zeros(2**m)
p0 = p0.at[0].set(1.)
pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = state, transpose = False)

#jax_lik._grad_coupled(theta, lam1, lam2, state, p0, latent_dist, latent_state, True)
inds = ssr.obs_inds(pTh1, state, latent_dist, obs_prim=True)
print(utils.trunk_states(state)[inds])
pTh1_obs = pTh1.at[inds].get()
#print(pTh1_obs)
#iwas = utils.ssr_obs_dist(pTh1, state, n, False)
#print(iwas)
#nk = pTh1_obs.sum()
#q = jnp.zeros_like(p0)
#pos = 2**(m-1) + inds.at[0].get()
#q = q.at[pos].set(1/nk)
#print(q)
print(utils.ssr_obs_dist(utils.trunk_states(state), state, n, True))
#print(pTh1_obs.shape)
# Actual gradient
#q = ssr.marg_transp(q, state, marg_met=False, marg_seeding=False)
#print(q)
#q = jax_lik.R_i_inv_vec(theta, q, lam1, state, transpose = True)
#g_1 = jax_lik.x_partial_Q_y(theta, q, pTh1, state)
#g_1

[0 1 1 1 0 1 1]
[1 1 1 1]
['0010000' '0110000' '0011000' '0111000' '0010010' '0110010' '0011010'
 '0111010' '0010001' '0110001' '0011001' '0111001' '0010011' '0110011'
 '0011011' '0111011']
2
3
2
['0010000' '0110000' '0011000' '0111000' '0010010' '0110010' '0011010'
 '0111010' '0010001' '0110001' '0011001' '0111001' '0010011' '0110011'
 '0011011' '0111011']


In [12]:
mat11 = np.array([[1,0,1,0],[0,1,0,1]])
mat10 = np.array([[1,1]])
mat1 = np.array([[1,0], [0,1]])
matC = np.kron(mat1, np.kron(mat10, np.kron(mat11, mat10)))
B = matC @ pTh1
#print(matC)
print(utils.trunk_states(state))
iwas = np.zeros_like(B)
iwas[-1] = 1/B[-1]
res1 = iwas.T @ matC
print(res1)
iwas2 = np.zeros_like(pTh1)
iwas2[inds] = 1/B[-1]
print(iwas2)

['0000000' '0100000' '0010000' '0110000' '0001000' '0101000' '0011000'
 '0111000' '0000010' '0100010' '0010010' '0110010' '0001010' '0101010'
 '0011010' '0111010' '0000001' '0100001' '0010001' '0110001' '0001001'
 '0101001' '0011001' '0111001' '0000011' '0100011' '0010011' '0110011'
 '0001011' '0101011' '0011011' '0111011']
[  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
 187.4078064 187.4078064   0.          0.        187.4078064 187.4078064
   0.          0.        187.4078064 187.4078064   0.          0.
 187.4078064 187.4078064]
[  0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.     187.4078 187.4078   0.       0.     187.4078 187.4078
   0.       0.     187.4078 187.4078   0.       0.     187.4078 187.4078]
